In [1]:
import os
os.chdir("..")
!pwd

/datasets/home/home-00/10/410/rcgonzal/DSC180Malware/m2v-adversarial-hindroid


In [2]:
from p_tqdm import p_umap
from stellargraph import StellarGraph
from stellargraph.data import UniformRandomMetaPathWalk
import pickle
import numpy as np
import pandas as pd
import json

In [3]:
outfolder = os.path.join('data/out/test-sample/')
graph_path = os.path.join(outfolder, 'graph.pkl')

with open(graph_path, 'rb') as file:
    g = pickle.load(file)
g

In [4]:
rw = UniformRandomMetaPathWalk(g)
app_nodes = list(g.nodes()[g.nodes().str.contains('app')])

In [5]:
walk_args = {
    'length': 60,
    'n': 100,
    'metapaths': [
        ["app", "api", "app"],
        ["app", "api", "method", "api", "app"],
        ["app", "api", "package", "api", "app"],
        ["app", "api", "package", "api", "method", "api", "app"],
        ["app", "api", "method", "api", "package", "api", "app"]
    ]
}

In [ ]:
# %%timeit
# walks = rw.run(app_nodes, **walk_args)

In [6]:
def run_walks(metapath):
    return rw.run(app_nodes, n=1, length=walk_args['length'], metapaths=[metapath])

In [7]:
metapaths = list(np.array(walk_args['metapaths']*walk_args['n'], dtype=object).flatten())
walks = np.concatenate(p_umap(run_walks, metapaths, num_cpus=16))

  0%|          | 0/500 [00:00<?, ?it/s]

In [11]:
walks.tolist()

[['app0',
  'api2461',
  'package53',
  'api2445',
  'method1920',
  'api2443',
  'app0',
  'api841',
  'package23',
  'api755',
  'method535',
  'api223',
  'app0',
  'api722',
  'package23',
  'api798',
  'method574',
  'api798',
  'app0',
  'api3130',
  'package60',
  'api3237',
  'method2421',
  'api3237',
  'app0',
  'api6094',
  'package83',
  'api6166',
  'method4419',
  'api224',
  'app2',
  'api9058',
  'package49',
  'api10533',
  'method7436',
  'api7144',
  'app3',
  'api730',
  'package0',
  'api3328',
  'method4558',
  'api6304',
  'app1',
  'api6467',
  'package184',
  'api6467',
  'method5808',
  'api6467',
  'app1',
  'api7128',
  'package40',
  'api21147',
  'method18034',
  'api21147',
  'app4',
  'api8487',
  'package15',
  'api7959',
  'method5484',
  'api7956'],
 ['app1',
  'api7459',
  'package85',
  'api14108',
  'method10996',
  'api6012',
  'app0',
  'api2931',
  'package60',
  'api3387',
  'method2697',
  'api3590',
  'app0',
  'api2475',
  'package53',
  'ap

In [ ]:
with open('test.json', 'w') as file:
    json.dump([1,2,3], file)

In [14]:
from gensim.models import Word2Vec

In [19]:
w2v_args = {
    "size": 128,
    "window": 5,
    "min_count": 0,
    "sg": 1,
    "workers": 16,
    "iter": 5
}

In [22]:
Word2Vec(walks.tolist(), **w2v_args)

In [21]:
walks

array([['app0', 'api2461', 'package53', ..., 'api7959', 'method5484',
        'api7956'],
       ['app1', 'api7459', 'package85', ..., 'api6433', 'method4592',
        'api6433'],
       ['app2', 'api8572', 'package40', ..., 'api12695', 'method9552',
        'api12695'],
       ...,
       ['app2', 'api8866', 'app3', ..., 'api17847', 'app4', 'api18030'],
       ['app3', 'api9707', 'app3', ..., 'api10166', 'app4', 'api2119'],
       ['app4', 'api16258', 'app4', ..., 'api13048', 'app4', 'api19681']],
      dtype='<U11')

In [ ]:
perf_stats = pd.DataFrame(data=[
                 [3, 2.3, .778],
                 [10, 8.09, 1.79],
                 [100, 82, 14.7]
             ],
            columns=['n', 'normal', 'parallel'])
perf_stats

In [ ]:
perf_stats.plot(kind='line', x='n')

In [ ]:
(perf_stats.normal / perf_stats.parallel).mean()